In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
import nltk
import pickle
import random
import argparse
import csv
import os
from numpy import dot
from numpy.linalg import norm
from gensim.models import FastText,Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
import nltk
from nltk import sent_tokenize,word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import model_from_json
import warnings
warnings.filterwarnings('ignore')

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks')  
for file in os.listdir('.'):
  print(file)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Untitled0.ipynb
Word2Vec.ipynb
vector_avg.ipynb
Doc2Vec.ipynb
news_cleaned_combined_ctext.txt
vector.pkl
newsCleaned_final.txt
news_cleaned_combined_ctext_final.txt
vector_summary.ipynb
text_token.txt
word2vec.model
newsCleaned_ctxt.txt
newsCleaned.txt
bbc_pkl
bbc_ft.bin
model.ipynb
Untitled1.ipynb
bbc_short_sum_pkl
models
vector_text.ipynb
model_bbc_short_summary.ipynb
Untitled2.ipynb
weights_satisfied.hdf5
outputs.npy
inputs.npy
model.json
model.h5
model_bbc

In [0]:
allWords = []
MAX_SENTENCE_LEN = 35
EMBEDDING_SIZE = 100
# p is the index into our training data for where we are now.
p = 0
accuracy_print_interval = 5
# number of neurons in each layer
input_num_units = MAX_SENTENCE_LEN*EMBEDDING_SIZE
hidden_num_units = 1500
output_num_units = MAX_SENTENCE_LEN
epochs = 250
batch_size = 32
learning_rate = 0.01
model_ft = FastText.load('bbc_ft.bin')

In [0]:
def clean_str(string):
    string = re.sub(r"\'s", "", string)
    string = re.sub(r"\'ve", "", string)
    string = re.sub(r"n\'t", "", string)
    string = re.sub(r"\'re", "", string)
    string = re.sub(r"\'d", "", string)
    string = re.sub(r"\'ll", "", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"'", "", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[0-9]\w+|[0-9]","", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\n",'',string)
    string = re.sub(r"  "," ",string)
    return string.strip().lower()
  
def extract_sentences_from_paragraph(paragraph):
    sentences = sent_tokenize(paragraph)
    sent_list=[]
    for s in sentences:
      if(len(s.strip()) != 0):
        s=clean_str(s)
        new_s=' '
        for w in word_tokenize(s):
          new_s.join(lemmatizer.lemmatize(w))
        sent_list.append(s)
    return sent_list
    #return [clean_str(s) for s in sentences if len(s.strip()) != 0]

In [0]:
def make_array_vectorize(text):
  texts=[]
  texts.append(extract_sentences_from_paragraph(text))
  nested_list_len = lambda x: sum(len(list) for list in x)
  source_text_vectors = np.zeros((nested_list_len(texts), 3500))
  vec_idx = 0
  if(type(texts[0]) == list):
      for i in range(len(texts)):
          sentences = texts[i]
          sentences_container = []
          # Get text vector
          for s in sentences:
              sentence_vector = np.array([])
              s=remove_stopwords(strip_punctuation(strip_non_alphanum(str(s).lower())))
              s=clean_str(s)
              for w in word_tokenize(s):
                  w=lemmatizer.lemmatize(w)
                  if(model_ft.__contains__(w)==False):
                    continue
                  if(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                      sentence_vector = np.append(sentence_vector, model_ft[w])
                  else:
                      break
              while(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                  sentence_vector = np.append(sentence_vector,np.zeros(EMBEDDING_SIZE))
              source_text_vectors[vec_idx] = sentence_vector
              vec_idx+=1
  return (source_text_vectors)

In [6]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
text='Tacit approval from the White House for the weaker greenback, which could help counteract huge deficits . European single currency has shot up to successive all-time highs against the dollar over the past few months. she is in good form.Tacit approval from the White House for the weaker greenback, which could help counteract huge deficits . European single currency has shot up to successive all-time highs against the dollar over the past few months. she is in good form.Tacit approval from the White House for the weaker greenback, which could help counteract huge deficits . European single currency has shot up to successive all-time highs against the dollar over the past few months. she is in good form.'
words=text.split(' ')
print(len(words))
clean=' '
if(len(words)>35):
  text=remove_stopwords(text)
  words=text.split(' ')
  print(len(text.split()))
  if(len(words)>35):
    words=words[:35]
    print(len(words))
    print(' '.join(w for w in words))
else:
  print(text)
    
    
# arr_text=extract_sentences_from_paragraph(text)
# for sent is 
def make_text_35(text):
  words=text.split(' ')
  if(len(words)>35):
    text=remove_stopwords(text)
    words=text.split(' ')
    if(len(words)>35):
      words=words[:35]
    return (clean.join(w for w in words))
  else:
    return(text)

118
67
35
Tacit approval White House weaker greenback, help counteract huge deficits . European single currency shot successive all-time highs dollar past months. good form.Tacit approval White House weaker greenback, help counteract huge deficits . European single


In [12]:
text="Sorting the arrays indexes by their associated probability/'s in descending order.Grabbing the first two class label indices which are thus the top-2 predictions from our network.How is this done is what we will learn.My name is adotya jain. I study is pesu"
print(extract_sentences_from_paragraph(text))
vec=make_array_vectorize(text)
print(vec.shape)
print(vec)

['sorting the arrays indexes by their associated probability in descending order grabbing the first two class label indices which are thus the top predictions from our network how is this done is what we will learn my name is adotya jain', 'i study is pesu']
(2, 3500)
[[ 0.2001051   0.34322619 -0.39303607 ...  0.          0.
   0.        ]
 [ 0.01239833  0.17048369 -0.3352603  ...  0.          0.
   0.        ]]


In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam')
results=loaded_model.predict(vec)
for res in results:
  word_idx=sorted(range(len(res)), key=lambda i: res[i], reverse=True)[:15]#making top 15 values as 1
  for i in word_idx:
    res[i]=1
print(results)

Instructions for updating:
Colocations handled automatically by placer.
Loaded model from disk
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  2.6960557e-03 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.2376882e-05 2.0519286e-08 2.7548802e-10 2.5186222e-07
  1.2968289e-17 2.5110858e-17 1.5852445e-14 1.0220711e-14 2.1602152e-14
  3.0502991e-14 3.1671904e-14 1.3987813e-14 3.6845811e-14 4.3077195e-14
  2.9193167e-14 3.2400603e-14 3.3685707e-14 3.6730242e-14 4.0248973e-14]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 5.6972341e-13 1.2738887e-11 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 3.9188086e-13 1.0000000e+00 4.6353714e-13
  3.2648409e-12 2.9304985e-13 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.8294773e-14 2.8019123e-13 3.9337335e-17 5.6703509e-16
  3.1917665e-16 2.4745390e-16 2.4274919e

In [9]:
# text = "The European single currency has shot up to successive all-time highs against the dollar over the past few months."
# text='Tacit approval from the White House for the weaker greenback, which could help counteract huge deficits . European single currency has shot up to successive all-time highs against the dollar over the past few months.'
# print(extract_sentences_from_paragraph(text))
text="The European single currency has shot up to successive all-time highs against the dollar over the past few months. Tacit approval from the White House for the weaker greenback, which could help counteract huge deficits, has helped trigger the move. But now Europe says the euro has had enough, and Asia must now share some of the burden."
arr_text=extract_sentences_from_paragraph(text)
vec=make_array_vectorize(text)
print(vec.shape)
results=loaded_model.predict(vec)
# ck=results.copy()
for res in results:
  word_idx=sorted(range(len(res)), key=lambda i: res[i], reverse=True)[:15]
  for i in word_idx:
    res[i]=1
# for res in results:
#   for sent in sent_tokenize(text):
# #     print(sent)
#     print( " ".join([word for idx,word in enumerate(sent.split(" ")) if(res[idx] == 1)])) 
for res,sent in zip(results,arr_text):
  sent=make_text_35(sent)
  print( " ".join([word for idx,word in enumerate(sent.split(" ")) if(res[idx] == 1)])) 

(3, 3500)
the european single currency has shot up to successive all time over the past few
tacit approval from the white house for the weaker greenback which could help deficits helped
but now europe says the euro has had the burden


In [10]:
# # Import BeautifulSoup from bs4
# from bs4 import BeautifulSoup


# # Create a BeautifulSoup object from the HTML
# soup = BeautifulSoup('https://machinelearningmastery.com/clean-text-machine-learning-python/', "html5lib")
# type(soup)
# bs4.BeautifulSoup
# text = soup.get_text()
# print(text)
# !pip install html2text
# import urllib.request
# html = urllib.request.urlopen('https://machinelearningmastery.com/clean-text-machine-learning-python/')
# soup = BeautifulSoup(html)
# data = soup.findAll(text=True)
# print(data)

# import html2text
# print(html2text.html2text(html))
# from goose import Goose
# g = Goose()
# article = g.extract('https://machinelearningmastery.com/clean-text-machine-learning-python')

# # process/store ...
# print(article.cleaned_text)
import urllib.request
import json
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

# url = "https://www.reuters.com/article/us-uber-ceo-idUSKBN19C0G6"
url="https://contentxtractor.com/api/v1/?a=grab&key=12jopu98&url=https://www.joe.co.uk/fitness-health/how-to-train-like-an-olympian-and-get-the-ultimate-lean-phyqiue-79681"
headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need
output = json.loads(data)
print (output)
print(output['data']['text'])

def test_with_url(url):
  url='https://contentxtractor.com/api/v1/?a=grab&key=12jopu98&url='+str(url)
  headers={'User-Agent':user_agent,} 
  request=urllib.request.Request(url,None,headers) #The assembled request
  response = urllib.request.urlopen(request)
  data = response.read() # The data u need
  output = json.loads(data)
  print (output)
  print(output['data']['text'])
  text=output['data']['text']
  arr_text=extract_sentences_from_paragraph(text)
  print(len(arr_text))
  vec=make_array_vectorize(text)
  print(vec.shape)
  results=loaded_model.predict(vec)
  for res in results:
    word_idx=sorted(range(len(res)), key=lambda i: res[i], reverse=True)[:15]
    for i in word_idx:
      res[i]=1
  sent_sum=' '
  for res,sent in zip(results,arr_text):
    sent=str(make_text_35(sent))
    s_indv=' '.join([word for idx,word in enumerate(sent.split(" ")) if(res[idx] == 1)])
    sent_sum=str(sent_sum)+str(s_indv)+str(',')
    print(s_indv)
  return sent_sum
     

{'data': {'title': 'How to train like an Olympian and get the ultimate lean physique', 'text': "If you've not noticed, the Olympics are on and the athletes are all in incredible shape. But right at the top of the tree in terms of body composition are the Olympic sprinters (yes we know - the weightlifters and gymnasts are pretty damn hench too). The likes of 100m king Usain Bolt and Team GB sprinterHarry Aikines-Aryeeteyare lean, muscular and shredded all year round. It's the physique that so many of us are trying to achieve slogging our guts out in the gym - and it's trainingthe bigger fast-twitch muscle fibres for maximal power helps build them into physical specimens. But if you want to look like an Olympian then you're probably going to have to train and eat like one. Sarah Claxtonis a former Olympic hurdlerwho now uses her knowledge of training and nutrition to get people in the best shape of their lives at London's Embody Fitness. The ex-English athlete specialises in the 100m hur

In [11]:
# text=output['data']['text']
# arr_text=extract_sentences_from_paragraph(text)
# print(len(arr_text))
# vec=make_array_vectorize(text)
# print(vec.shape)
# results=loaded_model.predict(vec)
# for res in results:
#   word_idx=sorted(range(len(res)), key=lambda i: res[i], reverse=True)[:15]
#   for i in word_idx:
#     res[i]=1
# for res,sent in zip(results,arr_text):
#   sent=str(make_text_35(sent))
#   print(" ".join([word for idx,word in enumerate(sent.split(" ")) if(res[idx] == 1)]))
# url='https://www.reuters.com/article/us-uber-ceo-idUSKBN19C0G6'
# url='http://news.bbc.co.uk/2/hi/technology/4249437.stm'
url='http://news.bbc.co.uk/2/hi/business/4236959.stm'
# test_with_url(url)
text_summary=test_with_url(url)

{'data': {'title': '', 'text': 'Blockbusters like \'Alexander\' failed to conquer at the box office Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Mixed fortunes Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the o

In [33]:
!pip install rouge

In [83]:
with open('bbc_pkl/texts.pkl', 'rb') as pickle_file:
    source=pickle.load(pickle_file)
with open('bbc_pkl/summary.pkl', 'rb') as pickle_file:
    target=pickle.load(pickle_file)
    
print((source[0]))
print(type(source[1]))

['quarterly profits at us media giant timewarner jumped to for the three months to december from year earlier', 'the firm which is now one of the biggest investors in google benefited from sales of high speed internet connections and higher advert sales', 'timewarner said fourth quarter sales rose to from', 'its profits were buoyed by one off gains which offset a profit dip at warner bros and less users for aol', 'time warner said on friday that it now owns of search engine google', 'but its own internet business aol had has mixed fortunes', 'it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters', 'however the company said aol underlying profit before exceptional items rose on the back of stronger internet advertising revenues', 'it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers for high speed broadband', 'timewarner also has to restate and resu

In [41]:
import rouge
evaluator = rouge.Rouge(metrics=['rouge-1','rouge-2'])
scores = evaluator.get_scores(target[0], text_summary)
print(scores)

[{'rouge-1': {'f': 0.5491803233539372, 'p': 0.7976190476190477, 'r': 0.41875}, 'rouge-2': {'f': 0.36723163395496194, 'p': 0.5462184873949579, 'r': 0.2765957446808511}}]


In [79]:
print(source[:2])

[['quarterly profits at us media giant timewarner jumped to for the three months to december from year earlier', 'the firm which is now one of the biggest investors in google benefited from sales of high speed internet connections and higher advert sales', 'timewarner said fourth quarter sales rose to from', 'its profits were buoyed by one off gains which offset a profit dip at warner bros and less users for aol', 'time warner said on friday that it now owns of search engine google', 'but its own internet business aol had has mixed fortunes', 'it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters', 'however the company said aol underlying profit before exceptional items rose on the back of stronger internet advertising revenues', 'it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers for high speed broadband', 'timewarner also has to restate and res

In [0]:
def test_vectorize(texts):
#   print(texts)
#   print(len(texts))
  nested_list_len = lambda x: sum(len(list) for list in x)
  source_text_vectors = np.zeros((nested_list_len(texts), 3500))
  vec_idx = 0
  if(type(texts[0]) == list):
      for i in range(len(texts)):
          sentences = texts[i]
          sentences_container = []
          # Get text vector
          for s in sentences:
              sentence_vector = np.array([])
              s=remove_stopwords(strip_punctuation(strip_non_alphanum(str(s).lower())))
              s=clean_str(s)
              for w in word_tokenize(s):
                  w=lemmatizer.lemmatize(w)
                  if(model_ft.__contains__(w)==False):
                    continue
                  if(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                      sentence_vector = np.append(sentence_vector, model_ft[w])
                  else:
                      break
              while(len(sentence_vector) < MAX_SENTENCE_LEN*EMBEDDING_SIZE):
                  sentence_vector = np.append(sentence_vector,np.zeros(EMBEDDING_SIZE))
              source_text_vectors[vec_idx] = sentence_vector
              vec_idx+=1
  return (source_text_vectors)

In [104]:
print(test_vectorize(source[0]).shape)

['quarterly profits at us media giant timewarner jumped to for the three months to december from year earlier', 'the firm which is now one of the biggest investors in google benefited from sales of high speed internet connections and higher advert sales', 'timewarner said fourth quarter sales rose to from', 'its profits were buoyed by one off gains which offset a profit dip at warner bros and less users for aol', 'time warner said on friday that it now owns of search engine google', 'but its own internet business aol had has mixed fortunes', 'it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters', 'however the company said aol underlying profit before exceptional items rose on the back of stronger internet advertising revenues', 'it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aol existing customers for high speed broadband', 'timewarner also has to restate and resu

In [0]:
def find_rouge(text,org_sum_text):
#   arr_text=extract_sentences_from_paragraph(text)
  vec=test_vectorize(text)
#   print(len(vec))
  results=loaded_model.predict(vec)
  for res in results:
    word_idx=sorted(range(len(res)), key=lambda i: res[i], reverse=True)[:15]
    for i in word_idx:
      res[i]=1
  sent_sum=' '
  for res,sent in zip(results,text):
    sent=str(make_text_35(sent))
    s_indv=' '.join([word for idx,word in enumerate(sent.split(" ")) if(res[idx] == 1)])
    sent_sum=str(sent_sum)+str(s_indv)+str(',')
#     print(s_indv)
  #now computing rougue metric
#   print(sent_sum)
  evaluator = rouge.Rouge(metrics=['rouge-1','rouge-2'])
  scores = evaluator.get_scores(org_sum_text, sent_sum)
  return scores[0]['rouge-1']['f']

In [0]:
def find_rouge_dataset(source,target):
  score=[]
  for i in range(len(source)):
    print(i)
    score.append(find_rouge(source[i],target[i]))
  return score

In [153]:
scores=find_rouge_dataset(source,target)
#[{'rouge-1': {'f': 0.49107142388392866, 'p': 0.6547619047619048, 'r': 0.39285714285714285}, 'rouge-2': {'f': 0.24761904291781311, 'p': 0.3277310924369748, 'r': 0.1989795918367347}}]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [154]:
import matplotlib.pyplot as plt
import statistics as st
print(max(scores),st.mean(scores),st.stdev(scores),min(scores),len(scores))


0.6966292084949993 0.5532263470885768 0.04438536530310459 0.1999999980246914 2225
